# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,275.98,82,99,15.16,TF,1726456809
1,1,albany,42.6001,-73.9662,293.59,70,39,1.81,US,1726456810
2,2,grytviken,-54.2811,-36.5092,270.82,89,99,6.29,GS,1726456811
3,3,nishinoomote,30.7333,131.0000,303.46,84,40,5.66,JP,1726456812
4,4,yuzhno-kurilsk,44.0306,145.8556,291.45,57,1,5.71,RU,1726456813


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
city_data_df_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    color = "City",
    alpha=0.5
)

# Display the map
city_data_df_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal weather condition
# Temperature between 10 and 25 degree C.
# Humidity less than equal to 30
# Wind Speed less than equal to 5 m/s
# Cloudiness less than equal to 25

mask = (((city_data_df["Max Temp"] >= 10.0) | (city_data_df["Max Temp"] <= 25.0)) 
        & (city_data_df["Humidity"] <= 30) 
        & (city_data_df["Wind Speed"] <= 5.0)
        & (city_data_df["Cloudiness"] <= 25))

city_data_df_ideal = city_data_df[mask]

# Drop any rows with null values
city_data_df_ideal.dropna()

# Display sample data
city_data_df_ideal.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,port augusta,-32.5000,137.7667,292.12,26,0,4.86,AU,1726456930
191,191,dumas,35.8656,-101.9732,299.23,27,0,3.09,US,1726457039
227,227,i-n-salah,27.1935,2.4607,306.14,18,0,3.99,DZ,1726457079
239,239,hami,42.8000,93.4500,295.91,18,2,1.28,CN,1726457091
277,277,aral,40.5156,81.2636,297.29,26,12,2.16,CN,1726457136
282,282,luziania,-16.2525,-47.9503,294.59,30,0,2.85,BR,1726457141
302,302,recreo,-29.2818,-65.0610,289.05,30,0,4.66,AR,1726457165
315,315,barcelos,41.5388,-8.6151,291.31,27,0,3.18,PT,1726457180
479,479,ad dawadimi,24.5077,44.3924,298.79,24,0,1.21,SA,1726457361
538,538,lata,40.1629,-8.3327,287.82,26,0,2.22,PT,1726457428


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
df_columns = ["City","Country","Lat","Lng","Humidity"]
hotel_df = city_data_df_ideal[df_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
95,port augusta,AU,-32.5000,137.7667,26,
191,dumas,US,35.8656,-101.9732,27,
227,i-n-salah,DZ,27.1935,2.4607,18,
239,hami,CN,42.8000,93.4500,18,
277,aral,CN,40.5156,81.2636,26,
282,luziania,BR,-16.2525,-47.9503,30,
302,recreo,AR,-29.2818,-65.0610,30,
315,barcelos,PT,41.5388,-8.6151,27,
479,ad dawadimi,SA,24.5077,44.3924,24,
538,lata,PT,40.1629,-8.3327,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["lat"] = latitude
    params["lon"] = longitude

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port augusta - nearest hotel: Pampas Motel
dumas - nearest hotel: Restwell Motel
i-n-salah - nearest hotel: باجودة
hami - nearest hotel: Хами
aral - nearest hotel: No hotel found
luziania - nearest hotel: No hotel found
recreo - nearest hotel: Hotel Sarmiento
barcelos - nearest hotel: Hotel Dom Nuno
ad dawadimi - nearest hotel: No hotel found
lata - nearest hotel: Coimbra Green Hostel


,City,Country,Lat,Lng,Humidity,Hotel Name
95,port augusta,AU,-32.5000,137.7667,26,Pampas Motel
191,dumas,US,35.8656,-101.9732,27,Restwell Motel
227,i-n-salah,DZ,27.1935,2.4607,18,باجودة
239,hami,CN,42.8000,93.4500,18,Хами
277,aral,CN,40.5156,81.2636,26,No hotel found
282,luziania,BR,-16.2525,-47.9503,30,No hotel found
302,recreo,AR,-29.2818,-65.0610,30,Hotel Sarmiento
315,barcelos,PT,41.5388,-8.6151,27,Hotel Dom Nuno
479,ad dawadimi,SA,24.5077,44.3924,24,No hotel found
538,lata,PT,40.1629,-8.3327,26,Coimbra Green Hostel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)